In [1]:
from sklearn.datasets import make_classification
import torch

In [2]:
# Step 1 create a synthetic classification dataset using sklearn

X, y = make_classification(
    n_samples=10,       # number of samples
    n_features=2,       # number of features
    n_informative=2,    # number of informative features
    n_redundant=0,      # number of redundant features
    n_classes=2,        # number of classes
    random_state=42     # for reproducibility
)

In [3]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [4]:
X.shape

(10, 2)

In [5]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [6]:
y.shape

(10,)

In [7]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [8]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [9]:
from torch.utils.data import Dataset, DataLoader

In [10]:
class CustomDataset(Dataset): # inherit Dataset abstraction class
# Dataset Abstract Class: acts as a blueprint for how data should be accessed.
# it is used for using with DataLoader class, in other word it is being used as
# a helper class for DataLoader class.
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    # you can perform data transformation here before returning
    # for example: for image data image resize, black & white (gray) conversion, data augmentation.
    # for text data: converting to lower case, lemmataization, stop word removal.
    return self.features[index], self.labels[index]


In [11]:
dataset = CustomDataset(X, y)

In [12]:
len(dataset)

10

In [13]:
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [14]:
# DataLoader class: used for mini batch training. it converts dataset into a
# iterable format and changes batch dataset into a n number of batch sizes mini batches.
# After that mini batches used to train a model.
# Samplers: DataLoader class has sampler which determines the way of selecting samples from
# the dataset while loading. "shuffle = True" means randomly sampling and "Shuffle = False"
# means sequential sampling. Also, we can do custom sampling using Sampler.
# Parallel working: Dataloader class parallelly create mini batches to make it work faster.
# for that this class has parameters called "num_workers", "pin_memory" (if gpu used),
# "prefetch_factor".
# Collate function: a function in Dataloader class that specifies how to combine a list
# of sample (mini batch) from the dataset.
# drop_last: parameter in Dataloader class that drops last uneven datasamples after
# creating the even batches. 30 samples batch_size = 10 will create 3 batches and drop
# last 2 samples.
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [15]:
for batch_features, batch_labels in dataloader:
  print(batch_features)
  print(batch_labels)
  print("-"*50)

tensor([[-1.1402, -0.8388],
        [ 1.7273, -1.1858]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [-2.8954,  1.9769]])
tensor([1, 0])
--------------------------------------------------
tensor([[-0.9382, -0.5430],
        [-0.7206, -0.9606]])
tensor([1, 0])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [-0.5872, -1.9717]])
tensor([0, 0])
--------------------------------------------------
tensor([[1.7774, 1.5116],
        [1.8997, 0.8344]])
tensor([1, 1])
--------------------------------------------------


# **PyTorch Training Pipeline with Dataset and DataLoader**

In [16]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [18]:
df.shape

(569, 33)

In [19]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [20]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [21]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[: , 1:], df.iloc[: , 0], test_size=0.2)

In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
X_train

array([[ 1.37816649,  0.48999165,  1.3202627 , ...,  1.01234224,
        -0.54651279, -0.98676931],
       [-1.07906703, -0.41788364, -1.07479742, ..., -1.17657541,
        -0.03373821, -0.31616745],
       [ 0.62468985,  0.62510625,  0.6616723 , ..., -0.12193871,
        -0.24725949, -0.23701445],
       ...,
       [-0.83165679, -1.07449318, -0.86372124, ..., -1.06421739,
        -1.59632573, -0.58880559],
       [-0.75855831, -0.15002487, -0.75941034, ..., -0.41666832,
         0.27683819, -0.88013262],
       [-0.72200907, -0.76159622, -0.74877472, ..., -0.72148721,
        -0.06123716,  0.20822121]])

In [24]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [25]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [26]:
X_train_tensor.shape

torch.Size([455, 30])

In [27]:
y_train_tensor.shape

torch.Size([455])

In [28]:
from torch.utils.data import Dataset, DataLoader

In [29]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [30]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [31]:
train_dataset[10]

(tensor([ 0.8862,  0.6701,  0.8294,  0.7878,  0.0753, -0.2519,  0.0403,  0.4533,
          0.2387, -0.9831,  0.0697, -0.8793, -0.0054,  0.1194, -0.9313, -0.7456,
         -0.3643, -0.2970, -1.1309, -0.8151,  1.1216,  0.6083,  1.0251,  0.9885,
          1.0162,  0.0397,  0.5637,  1.3104,  0.5292, -0.4475]),
 tensor(1.))

In [32]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [34]:
# defining the model class
class MySimpleNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()

    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)

    return out


In [37]:
# important parameters
learning_rate = 0.1
epochs = 25

In [36]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

In [38]:
# training pipeline
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.reshape(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backword pass
    loss.backward()

    # parameter update
    optimizer.step()

  # print loss in each epoch
  print(f"Epoch: {epoch+1}, Loss:{loss.item()}")

Epoch: 1, Loss:0.2886669635772705
Epoch: 2, Loss:0.14823700487613678
Epoch: 3, Loss:0.07052253931760788
Epoch: 4, Loss:0.044836435467004776
Epoch: 5, Loss:0.48955339193344116
Epoch: 6, Loss:0.1542579084634781
Epoch: 7, Loss:0.04246426746249199
Epoch: 8, Loss:0.11280321329832077
Epoch: 9, Loss:0.2961491644382477
Epoch: 10, Loss:0.12779217958450317
Epoch: 11, Loss:0.024437207728624344
Epoch: 12, Loss:0.06282158941030502
Epoch: 13, Loss:0.15458358824253082
Epoch: 14, Loss:0.007580541539937258
Epoch: 15, Loss:0.12505684792995453
Epoch: 16, Loss:0.011019651778042316
Epoch: 17, Loss:0.05660957843065262
Epoch: 18, Loss:0.012789765372872353
Epoch: 19, Loss:0.13270913064479828
Epoch: 20, Loss:0.021114084869623184
Epoch: 21, Loss:0.16499578952789307
Epoch: 22, Loss:0.02899928204715252
Epoch: 23, Loss:0.010918148793280125
Epoch: 24, Loss:0.011269393377006054
Epoch: 25, Loss:0.01650167629122734


# **Evaluation**

In [40]:
# model evaluation
model.eval()  # set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    y_pred = model(batch_features)
    y_pred = (y_pred>0.5).float()  # convert probabilities into binary prediction

    # calculate accuracy for the current batch
    batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
    accuracy_list.append(batch_accuracy)

# calculate overall accuracy
overall_accuracy = sum(accuracy_list)/ len(accuracy_list)
print(f"Accuracy: {overall_accuracy:.4f}")

Accuracy: 0.9549
